Some of the imports are not useful yet

In [1]:
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle 
from yaafelib import *

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Download GTZAN dataset once

In [3]:
def downloadGTZAN():
    filename = 'genres.tar.gz'
    if not os.path.exists(filename):
        filename, _ = urlretrieve('http://opihi.cs.uvic.ca/sound/genres.tar.gz', filename)
    else:
        print('File ' + filename + ' exists')
    
    return filename
filename = downloadGTZAN()

Extract files from .tar.gz

In [4]:
num_classes = 10

def extract(filename):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root):
        print('File %s already extracted' % (root))
    else:
        print('Extracting %s ' % filename)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))
                    if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders not found.' % (num_classes))
    print(data_folders)
    return data_folders

folders = extract(filename)

Extracting genres.tar.gz 
['genres/blues', 'genres/classical', 'genres/country', 'genres/disco', 'genres/hiphop', 'genres/jazz', 'genres/metal', 'genres/pop', 'genres/reggae', 'genres/rock']


In [31]:
def get_features(audio_file):
    if os.path.exists(audio_file):
        print('Getting features from ' + audio_file)
    else:
        raise Exception('File ' + audio_file + ' not found')
    fp = FeaturePlan(sample_rate=22050, normalize=1)
    # Features that seems to be most often used, so they are good to start with.
    fp.addFeature('mfcc: MFCC')
    fp.addFeature('zcr: ZCR')
    fp.addFeature('spectral_shape: SpectralShapeStatistics')
    fp.addFeature('magnitude_spectrum: MagnitudeSpectrum')
    df = fp.getDataFlow()
    engine = Engine()
    engine.load(df)
    afp = AudioFileProcessor()
    afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
    afp.processFile(engine, audio_file)
    engine.flush()
    feats = engine.readAllOutputs()
    return feats

def load_genre(folder):
    print('Loading genre from folder ' + folder)
    samples = os.listdir(folder)
    dataset =[]
    for sample in os.listdir(folder):
        sample_file = os.path.join(folder, sample)
        if sample.endswith('.au'):
            features = get_features(sample_file)
            dataset.append(features)
    return dataset

def pickle(data_folders):
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
        if os.path.exists(set_filename):
            print('%s already pickled' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_genre(folder)
            with open(set_filename, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
    return dataset_names

pickle(folders)

Pickling genres/blues.pickle.
Loading genre from folder genres/blues
Getting features from genres/blues/blues.00004.au
Getting features from genres/blues/blues.00086.au
Getting features from genres/blues/blues.00009.au
Getting features from genres/blues/blues.00022.au
Getting features from genres/blues/blues.00078.au
Getting features from genres/blues/blues.00066.au
Getting features from genres/blues/blues.00005.au
Getting features from genres/blues/blues.00057.au
Getting features from genres/blues/blues.00056.au
Getting features from genres/blues/blues.00019.au
Getting features from genres/blues/blues.00071.au
Getting features from genres/blues/blues.00097.au
Getting features from genres/blues/blues.00014.au
Getting features from genres/blues/blues.00099.au
Getting features from genres/blues/blues.00080.au
Getting features from genres/blues/blues.00094.au
Getting features from genres/blues/blues.00093.au
Getting features from genres/blues/blues.00065.au
Getting features from genres/bl

['genres/blues.pickle',
 'genres/classical.pickle',
 'genres/country.pickle',
 'genres/disco.pickle',
 'genres/hiphop.pickle',
 'genres/jazz.pickle',
 'genres/metal.pickle',
 'genres/pop.pickle',
 'genres/reggae.pickle',
 'genres/rock.pickle']